In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import json
from src.algorithms import hoeffding_bound

In [ ]:
CHEMICAL_ACCURACY = 1.6e-3
delta = 0.1

TODO: exemplify EBS by running it on one parameter choice $D$
===

In [ ]:
# load params
params = np.loadtxt("h2_hamiltonians/h2_parameter.csv",skiprows=1,delimiter=",").T
R, offsets, params = params[0], params[1], params[2:].T
eps_bern = CHEMICAL_ACCURACY

Obtaining energy samples for arbitrary Hamiltonians
---

The high-level function `get_estimator` implements algorithm 1 and is then called by algorithm 3

In [ ]:
from dep.shadow_grouping import get_estimator

In [ ]:
from dep.shadow_grouping import get_pauli_list, Hamiltonian, char_to_int
from dep.molecules import H2

In [ ]:
decomposition = get_pauli_list(H2)
offset = decomposition[0,1]
obs_str = decomposition[1:,0]
obs = np.array([[char_to_int[o] for o in obs] for obs in obs_str])
weights = decomposition[1:,1].astype(float)

In [ ]:
# calculate groundstate and its energy
H = Hamiltonian(weights,obs_str)
ham = H.SummedOp().to_matrix()
E_numerics, groundstate = H.ground(sparse=False)
groundstate = groundstate.real

In [ ]:
estimator = get_estimator(weights,obs.copy(),offset,groundstate)
samples = estimator.get_samples(1000)
samples.shape, samples.mean(), E_numerics + offset

This way, samples for the state's energy can be readily prepared.
We follow this procedure for all energy sample calculations, but only provide the processed data in the following.
Creating these data naturally is the most resource intensive step of our numerical results.

Recreate figure 1
===

In [ ]:
folder = "data/H2_data/"

E_GS     = np.zeros_like(R)
E_est    = np.zeros_like(R)
abs_diff = np.zeros_like(R)
abs_var  = np.zeros_like(R)
N_EBS    = np.zeros_like(R)
N_EBSvar = np.zeros_like(R)
N_Hoeff  = np.zeros_like(R)

for i,dist in enumerate(R):
    file = "{}{}/groundstate_epsilon_0.0016.txt".format(folder,dist)
    energy, samplesEBS, samplesHoeff, estimate, estim_var = np.loadtxt(file,skiprows=1,unpack=True)
    E_GS[i]     = energy[0]
    E_est[i]    = np.mean(estimate)
    abs_diff[i] = np.abs(estimate-energy).mean()
    abs_var[i]  = np.abs(estimate-energy).std()
    N_EBS[i]    = np.mean(samplesEBS)
    N_EBSvar[i] = np.std(samplesEBS)
    N_Hoeff[i]  = samplesHoeff[0]

Figure 1a
---

In [ ]:
eps = eps_bern
#E_GS = np.loadtxt("E_GS.txt")

# energy curve
ax = plt.subplot(111)
plt.plot(R,E_GS,'k-')
plt.plot(R,E_est,'x',c = 'yellowgreen')
plt.ylabel(r"Energy $E(D)$ [Ha]",fontsize="x-large")
plt.xlabel(r'Interatomic distance $D\ [\AA]$',fontsize="x-large")
plt.yticks(fontsize="large")
plt.xticks(fontsize="large")
#plt.tick_params("x",labelbottom=False, length=0)
axins = ax.inset_axes(
    [0.25, 0.4, 0.7, 0.5],xlim=plt.xlim(), ylim=(0, eps/2), yticklabels=["0",r"$\frac{\epsilon}{4}$",r"$\frac{\epsilon}{2}$"],
    facecolor="white", yticks = [0,eps/4,eps/2]
)
axins.plot(R,abs_diff,"x", c="yellowgreen")
#axins.fill_between([0,3],eps,color="grey",alpha=0.2)
axins.text(1.5,0.375*eps,r"$|\hat E - E |$",ha="center",weight="bold",fontsize="xx-large")
axins.grid()
plt.grid()

plt.show()

Figure 1b
---

In [ ]:
ax = plt.subplot(111)
plt.semilogy()

plt.plot(R,N_Hoeff,"ro",label="Hoeffding")
plt.plot(R,N_EBS,"gx",label="EBS")
plt.errorbar(R,N_EBS,yerr=N_EBSvar,fmt="none")
plt.ylim(1e5,4e7)
plt.xlabel(r'Interatomic distance $D\ [\AA]$',fontsize="x-large")
plt.ylabel(r'No. of meas. rounds $N(D)$',fontsize="x-large")
plt.legend(fontsize="x-large",loc=3)
plt.yticks(fontsize="large")
plt.xticks(fontsize="large")
plt.grid()

#inset
axins = ax.inset_axes(
    [0.4, 0.67, 0.575, 0.3],xlim=plt.xlim(), ylim=(0, 0.4), yticklabels=["0","20","40"],
    facecolor="white", yticks = [0,0.2,0.4]
)
axins.plot(R,N_EBS/N_Hoeff,"gx")
axins.text(1.5,0.15,r"$\frac{N_\mathrm{EBS}}{N_\mathrm{Hoeff}}$",
           ha="center",fontsize="xx-large",bbox=dict(facecolor='white', edgecolor="white", alpha=0.7))
axins.grid()

plt.show()

Recreate Figure 2
===

In [ ]:
from src.algorithms import EBS
from src.algorithms import hoeffding_bound
from os import listdir
from os.path import isfile
from scipy.optimize import curve_fit

In [ ]:
def load_samples(filename):
    params = {}
    with open(filename, "r") as f:
        line = f.readline().strip()
        while line.find("#") >= 0:
            key, val = line.split("=")
            params[key[2:]] = float(val)
            line = f.readline().strip()
    samples = np.loadtxt(filename, skiprows=len(params.keys()))
    return samples, params

def fit_func(N,A,c):
    return A/N**c

$\displaystyle \epsilon(N) = \frac{A}{N^c} $

Figure 2a
---

In [ ]:
from src.algorithms import EBS_UnModded

In [ ]:
folder = "data/samples_for_EBS/"

In [ ]:
samples, params = load_samples(folder + "{}_molecule_{}_old_samples_0.txt".format("BeH2", "JW"))
R = params["R"]
actual_mean = params["mean"]
Ngroups  = params["Ngroups"]

In [ ]:
Ngroups

In [ ]:
samples2, _ = load_samples(folder + "{}_molecule_{}_old_samples_9.txt".format("BeH2", "JW"))
samples = np.append(samples,samples2)

In [ ]:
eps_vals = np.logspace(-3.5,-1,8)
N_hoeff = np.zeros(len(eps_vals),dtype=int)
N_EBS   = np.zeros_like(N_hoeff)
N_plot = np.logspace(4,11,100)

In [ ]:
for index,eps in enumerate(eps_vals):
    print("Going for eps\t=",eps)
    np.random.shuffle(samples)
    N_hoeff[index] = hoeffding_bound(delta, eps, R)

    ebs = EBS(epsilon=eps, delta=delta, range_of_rndvar=R, num_groups=Ngroups, N_min=1)

    ind = 0
    while ebs.cond_check():# and ebs.get_numsamples()*Ngroups < N_hoeff[index]:
        ebs.add_sample(samples[ind])
        ind += 1
        if ind == len(samples):
            print("Warning: had to reshuffle data!")
            ind = 0
            np.random.shuffle(samples)
    N_EBS[index] = ebs.get_numsamples()*Ngroups

In [ ]:
popt_hoeff, pcov_hoeff = curve_fit(fit_func,N_hoeff,eps_vals,p0=[100,0.5],bounds=((0,0),(np.inf,2)))
print("c_Hoeff = {:.2f} +/- {:.2f}".format(popt_hoeff[-1],np.sqrt(pcov_hoeff[-1,-1])))
popt_ebs, pcov_ebs     = curve_fit(fit_func,N_EBS,eps_vals,p0=[100,0.5],bounds=((0,0),(np.inf,2)))
print("c_EBS = {:.2f} +/- {:.2f}".format(popt_ebs[-1],np.sqrt(pcov_ebs[-1,-1])))

In [ ]:
plt.loglog(N_EBS,eps_vals,"ro",label="EBS\n$c={:.2f} \pm {:.2f}$".format(popt_ebs[-1],np.sqrt(pcov_ebs[-1,-1])))
plt.plot(N_plot,fit_func(N_plot,*popt_ebs),"r--")
plt.loglog(N_hoeff,eps_vals,"gx",label="Hoeffding\n$c=0.5$")
plt.plot(N_plot,fit_func(N_plot,*popt_hoeff),"g-")
#plt.plot(N_plot,STD*np.sqrt(2*np.log(20)/N_plot*Ngroups)+2*R*np.log(20)/3/N_plot*Ngroups,"k--",label="Bernstein")
plt.legend(fontsize="x-large")
plt.text(1e6,8e-3,r"$\epsilon(N) = \frac{A}{N^c} $",
               ha="center",fontsize="xx-large",bbox=dict(facecolor='white', edgecolor="white", alpha=0.7))
plt.fill_between(plt.xlim(),CHEMICAL_ACCURACY,color="grey",alpha=0.2)
plt.ylabel(r"Accurancy $\epsilon$ [Ha]",fontsize="x-large")
plt.xlabel(r'Total no. meas. rounds $N$',fontsize="x-large")
plt.yticks(fontsize="large")
plt.xticks(fontsize="large")
#plt.ylim(1e-4,2e-1)
#plt.xlim(1e5,1e11)
plt.grid()

plt.show()

¶Figure 2b
---

We will use preprocessed data, but the data generation procedure from samples is depicted below as well.

In [ ]:
available_molecules = ["H2", "H2_6-31g", "LiH", "BeH2", "H2O", "NH3"]
mappings = ["JW", "BK", "Parity"]
epsilon = CHEMICAL_ACCURACY
beta = 1.1

In [ ]:
savefile = 'data/bar_plot_data.txt'

In [ ]:
estimates = np.zeros(len(mappings)*len(available_molecules))
targets   = np.zeros_like(estimates)

if isfile(savefile):
    data = np.loadtxt(savefile, delimiter=',')
    arr_jw, arr_bk, arr_par, arr_höf = data
else:
    arr_jw = []
    arr_bk = []
    arr_par = []
    arr_höf = []

    for ind,mol in enumerate(available_molecules):
        samples_jw, params_jw = load_samples(
            folder + "{}_molecule_{}_samples_long.txt".format(mol, "JW"))
        samples_bk, params_bk = load_samples(
            folder + "{}_molecule_{}_samples_long.txt".format(mol, "BK"))
        samples_par, params_par = load_samples(
            folder + "{}_molecule_{}_samples_long.txt".format(mol, "Parity"))

        np.random.shuffle(samples_jw)
        np.random.shuffle(samples_bk)
        np.random.shuffle(samples_par)

        R_jw = params_jw["R"]
        R_bk = params_bk["R"]
        R_par = params_par["R"]

        actual_mean = params_jw["mean"]
        actual_mean = params_bk["mean"]
        actual_mean = params_par["mean"]
        
        targets[len(mappings)*ind:len(mappings)*(ind+1)] = actual_mean

        Ngroups_jw  = params_jw["Ngroups"]
        Ngroups_bk  = params_bk["Ngroups"]
        Ngroups_par = params_par["Ngroups"]

        tmin = hoeffding_bound(delta, epsilon, R_jw)
        arr_höf.append(tmin)

        ebs_jw  = EBS(epsilon=epsilon, delta=delta, range_of_rndvar=R_jw,
                      num_groups=Ngroups_jw, beta=beta, N_min=tmin//100//Ngroups_jw)
        ebs_bk  = EBS(epsilon=epsilon, delta=delta, range_of_rndvar=R_bk,
                      num_groups=Ngroups_bk, beta=beta, N_min=tmin//100//Ngroups_bk)
        ebs_par = EBS(epsilon=epsilon, delta=delta, range_of_rndvar=R_par,
                      num_groups=Ngroups_par, beta=beta, N_min=tmin//100//Ngroups_par)

        # JW
        ind_jw = 0
        while ebs_jw.cond_check():
            ebs_jw.add_sample(samples_jw[ind_jw])
            ind_jw += 1
            if ind_jw == len(samples_jw):
                print("Warning for {}-molecule (JW): had to reshuffle data!".format(mol))
                ind_jw = 0
                np.random.shuffle(samples_jw)
        arr_jw.append(ebs_jw.get_step()*Ngroups_jw)
        print("{}-molecule (JW): required 10^{:.1f} samples (10^{:.1f} max.)".format(
            mol,np.log10(arr_jw[-1]),np.log10(arr_höf[-1])
        ))
        estimates[len(mappings)*ind] = ebs_jw.get_estimate()

        # BK
        ind_bk = 0
        while ebs_bk.cond_check():
            ebs_bk.add_sample(samples_bk[ind_bk])
            ind_bk += 1
            if ind_bk == len(samples_bk):
                print("Warning for {}-molecule (BK): had to reshuffle data!".format(mol))
                ind_bk = 0
                np.random.shuffle(samples_bk)
        arr_bk.append(ebs_bk.get_step()*Ngroups_bk)
        print("{}-molecule (BK): required 10^{:.1f} samples (10^{:.1f} max.)".format(
            mol,np.log10(arr_bk[-1]),np.log10(arr_höf[-1])
        ))
        estimates[len(mappings)*ind+1] = ebs_bk.get_estimate()

        # Parity
        ind_par = 0
        while ebs_par.cond_check():
            ebs_par.add_sample(samples_par[ind_par])
            ind_par += 1
            if ind_par == len(samples_par):
                print("Warning for {}-molecule (Parity): had to reshuffle data!".format(mol))
                ind_par = 0
                np.random.shuffle(samples_par)
        arr_par.append(ebs_par.get_step()*Ngroups_par)
        print("{}-molecule (Parity): required 10^{:.1f} samples (10^{:.1f} max.)".format(
            mol,np.log10(arr_par[-1]),np.log10(arr_höf[-1])
        ))
        estimates[len(mappings)*ind+2] = ebs_par.get_estimate()
        

    np.savetxt(savefile, (arr_jw,arr_bk,arr_par,arr_höf), delimiter=',')

In [ ]:
width = 0.2
align = "center" # or edge
color_JW = "#ffa600"
color_BK = "#ff6361"
color_Pa = "#bc5090"
color_Ho = "#003f5c"

br1 = np.arange(len(arr_bk)) - 1.5*width
br2 = br1 + width
br3 = br2 + width
br4 = br3 + width

In [ ]:
plt.bar(br1, np.asarray(arr_bk), color = color_JW, width = width,
        edgecolor ='grey', label ='Jordan-Wigner', zorder=3, align=align)

plt.bar(br2, np.asarray(arr_jw), color = color_BK, width = width,
        edgecolor ='grey', label ='Bravyi-Kitaev', zorder=3, align=align)

plt.bar(br3, np.asarray(arr_par), color = color_Pa, width = width,
        edgecolor ='grey', label ='Parity', zorder=3, align=align)

plt.bar(br4, np.asarray(arr_höf), color = color_Ho, width = width,
        edgecolor ='grey', label ='Hoeffding', zorder=3, align=align)

plt.yscale('log')
#plt.xlabel('Molecule', fontsize = "xx-large")
plt.ylabel('Number of Samples', fontsize = "xx-large")
plt.xticks(np.arange(len(arr_jw)),
        available_molecules,fontsize="x-large")
plt.yticks(fontsize="x-large")
plt.legend(fontsize="large")
plt.grid(axis="y",zorder=1)

plt.show()

Recreate Figure 3 & 4
===

Imaginary time evolution, given a Hamiltonian $H$ and parameter $\beta$, evolves a quantum state $\vert \psi \rangle$ according to
$$\vert \psi \rangle \mapsto \vert \psi' \rangle = e^{-\beta H} \vert \psi \rangle \mapsto \frac{ \vert \psi' \rangle }{ \Vert \vert \psi' \rangle \Vert }$$
Repeating these two steps over and over yields the lowest excited state of $H$ that has non-zero overlap with the initial $\vert \psi \rangle$:
$$\lim_{\beta \to \infty} e^{-\beta H} \vert \psi \rangle = \vert E_{i^\ast} \rangle\,,\qquad \text{where} \qquad i^\ast = \min \{ i \mid \langle \psi \vert E_i \rangle \neq 0\}\,.$$


and we end up with the lowest excited state of $H$ that shares non-zero overlap with the initial $\vert \psi \rangle$ up to a global phase.

In [ ]:
from dep.shadow_grouping import get_pauli_list, Hamiltonian
from dep.molecules import H4
from numpy.linalg import eigh
from dep.imaginary_time_evolution import ImaginaryTimeEvolution
from dep.shadow_grouping import JordanWignerMapper, BravyiKitaevMapper, ParityMapper

In [ ]:
betas = np.logspace(0,3,20)
energies = np.zeros_like(betas)
fidelites = np.zeros_like(betas)

decomposition = get_pauli_list(H4,JordanWignerMapper)

offset = decomposition[0,1]
obs = decomposition[1:,0]
weights = decomposition[1:,1].astype(float)

In [ ]:
# calculate groundstate and its energy
H = Hamiltonian(weights,obs)
ham = H.SummedOp().to_matrix()
#E_numerics, groundstate = H.ground(sparse=False)
#groundstate = groundstate.real

vals, states =  eigh(ham)
states = states.real # the eigenstates are real-valued because the Hamiltonian is as well
E_numerics = vals[0]
groundstate = states[:,0]

In [ ]:
beta = 10
timeevo = ImaginaryTimeEvolution(weights,obs,beta)

timeevo.reset()
for i,beta in enumerate(betas):
    timeevo.reset()
    if i==0:
        print("Initial energy of",timeevo.energy)
    timeevo.set_beta(beta)
    timeevo.step()
    energies[i] = timeevo.energy
    fidelites[i] = timeevo.fidelity_with_groundstate
    print("beta",beta,"energy",timeevo.energy)

Figure 3
---

In [ ]:
plt.loglog(betas,np.abs(energies-E_numerics),"k-",label=r"$E(\beta)-E_{GS}$ [Ha]")
plt.loglog(betas,1-fidelites,color="gray",linestyle="dashed",label="Infidelity $1-F(\\beta)$")
plt.legend(fontsize="xx-large")
plt.xlim(1,1000)
plt.ylim(1e-10,2)
plt.yticks(fontsize="x-large")
plt.ylabel("Proximity measure",fontsize="xx-large")
plt.xticks(fontsize="x-large")
plt.xlabel("Inverse temperature $\\beta$",fontsize="xx-large")
plt.grid()

plt.text(2,3e-4,r"$\vert \beta \rangle \propto e^{-\beta H} \vert + \rangle^{\otimes n}$",fontsize=20)

plt.show()

Figure 4
---

In [ ]:
def json_load(filename):
    with open(filename,"r") as f:
        temp = json.load(f)
    temp_dict = {}
    for key,val in temp.items():
        if key=="-1":
            continue
        tup = tuple(temp["-1"][key])
        tup, index = tup[:-1], tup[-1]
        subdict = temp_dict.get(tup,{})
        subdict[index] = val
        temp_dict[tup] = subdict

    # clean up data further - get the values for epsilon and beta, respectively
    epsilons, betas = set(), set()
    for key in temp_dict:
        epsilons.add(key[2])
        betas.add(key[1])
    epsilons, betas = np.sort(list(epsilons)), np.sort(list(betas))
    
    # aggregate data for the same keys as np.array and store these in out-dict
    out = {}
    for key,val in temp_dict.items():
        out[key] = np.array([val[index] for index in range(len(val))])
    return out, epsilons, betas

In [ ]:
filename = "data/H4_data.json"
map_name = "JW"
colors  = ("red","blue","purple","cyan")
eps_ticks = 1.6*np.logspace(-1,-4,7)[::-1]

diction, epsilons, betas = json_load(filename)

In [ ]:
#borders = 1e-5,2e-1

plt.figure(figsize=(9,6))
ax = plt.subplot(111)
plt.semilogx()
for beta,color in zip(betas[::3],colors):
    Nshots = np.zeros_like(epsilons)
    Nhoeff = np.zeros_like(Nshots)
    stds = np.zeros_like(Nshots)
    for i,eps in enumerate(epsilons):
        vals = diction[(map_name,beta,eps)]
        Nshots[i] = vals[:,1].mean()
        Nhoeff[i] = vals[0,2]
    plt.plot(epsilons,Nhoeff/Nshots,color=color,label="beta={}".format(int(beta)))
#plt.plot(epsilons,Nhoeff,label="Hoeff")
plt.legend(fontsize="x-large",loc="lower left")
#plt.plot(borders,borders,"k--")
#plt.xlim(right=1e-2)
#plt.ylim(bottom=1e6)
plt.ylabel(r"$N_{Hoeff}\ /\ N_{EBS}$",fontsize="xx-large")
plt.xlabel("Normalized accuracy $\epsilon/\epsilon_{acc.}^{chem.}$",fontsize="xx-large")
plt.xticks(eps_ticks,["0.1","","1","","10", "", "100"],fontsize="x-large")
plt.yticks(fontsize="x-large")
plt.grid()

# second x axis on top
N0 = Nhoeff[epsilons == 0.0016][0]
Nvals = np.logspace(4,10,7)
N_ticks = np.sqrt(N0/Nvals)*0.0016
ax_top = ax.secondary_xaxis("top")
ax_top.set_xticks(N_ticks)
ax_top.set_xticklabels(["$10^4$","","$10^6$","","$10^8$", "", "$10^{10}$"],fontsize="x-large")
ax_top.set_xlabel(r"$N_{Hoeff}\ (\epsilon/\epsilon_{acc.}^{chem.})$",fontsize="xx-large")

if True:
    axins = ax.inset_axes(
        [0.475, 0.62, 0.5, 0.3], ylim=(0.06, 0.09),facecolor="white"#, yticks = [0,0.2,0.4], yticklabels=["0","20","40"]
    )
    variances = np.zeros_like(betas[::3])
    eps = epsilons[0]
    for i,beta in enumerate(betas[::3]):
        vals = diction[(map_name,beta,eps)]
        variances[i] = vals[:,-1].mean()
    axins.bar(np.arange(len(variances)),variances,color=colors, zorder=3)
    axins.set_ylabel(r"STD $\sigma$ [Ha]",fontsize="x-large")
    axins.set_xlabel("Inverse temperature $\\beta$",fontsize="x-large")
    axins.set_xticks(np.arange(len(variances)),["1", "10", "100", "100"],fontsize=15)
    axins.yaxis.set_tick_params(labelsize=15)
    axins.grid(axis="y",zorder=1)
    #axins.text(1.5,0.15,r"$\frac{N_\mathrm{EBS}}{N_\mathrm{Hoeff}}$",ha="center",fontsize="xx-large",bbox=dict(facecolor='white', edgecolor="white", alpha=0.7))
    #axins.grid(which="minor",axis="y")

plt.show()